In [37]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [38]:
import numpy as np
import pandas as pd

Mydata = np.array([[1 , 1, 1, 0, 0], #No
                [1 , 1, 1, 1, 0], #No
                [0 , 1, 1, 0, 1],
                [2 , 0, 1, 0, 1],
                [2 , 2, 0, 0, 1],
                [2 , 2, 0, 1, 0],#No
                [0 , 2, 0, 1, 1],
                [1 , 0, 1, 0, 0],#No
                [1 , 2, 0, 0, 1],
                [2 , 0, 0, 0, 1],
                [1 , 0, 0, 1, 1],
                [0 , 0, 1, 1, 1],
                [0 , 1, 0, 0, 1],
                [2 , 0, 1, 1, 0]])#No
Mydata = pd.DataFrame(Mydata, columns = ['Outlook', 'Tmp', 'Humidity', 'Windy', 'Play'])
Mydata

Outlook  Tmp  Humidity  Windy  Play
0         1    1         1      0     0
1         1    1         1      1     0
2         0    1         1      0     1
3         2    0         1      0     1
4         2    2         0      0     1
5         2    2         0      1     0
6         0    2         0      1     1
7         1    0         1      0     0
8         1    2         0      0     1
9         2    0         0      0     1
10        1    0         0      1     1
11        0    0         1      1     1
12        0    1         0      0     1
13        2    0         1      1     0

In [51]:
def ent(data):
    prob1 = pd.value_counts(data) / len(data)
    return sum(np.log2(prob1) * prob1 * (-1))

def gain(data,str1,str2):
    e1 = data.groupby(str1).apply(lambda x:ent(x[str2]))
    print(e1)
    p1 = pd.value_counts(data[str1]) / len(data[str1])
    e2 = sum(e1 * p1)
    print('Conditional Entropy of ',str1,e2)
    return ent(data[str2]) - e2

print('Entropy of Play',ent(Mydata['Play']))
print('-----------------------------------------')
print('IG Humidity',gain(Mydata,'Humidity','Play'))
print('-----------------------------------------')
print('IG Outlook',gain(Mydata,'Outlook','Play'))
print('-----------------------------------------')
print('IG Tmp',gain(Mydata,'Tmp','Play'))
print('-----------------------------------------')
print('IG Windy',gain(Mydata,'Windy','Play'))

Entropy of Play 0.9402859586706311
-----------------------------------------
Humidity
0    0.591673
1    0.985228
dtype: float64
Conditional Entropy of  Humidity 0.7884504573082896
IG Humidity 0.15183550136234159
-----------------------------------------
Outlook
0    0.000000
1    0.970951
2    0.970951
dtype: float64
Conditional Entropy of  Outlook 0.6935361388961918
IG Outlook 0.24674981977443933
-----------------------------------------
Tmp
0    0.918296
1    1.000000
2    0.811278
dtype: float64
Conditional Entropy of  Tmp 0.9110633930116763
IG Tmp 0.02922256565895487
-----------------------------------------
Windy
0    0.811278
1    1.000000
dtype: float64
Conditional Entropy of  Windy 0.8921589282623617
IG Windy 0.04812703040826949


# H6751 Hands-on Session 3 Text Preprocessing II

1. This lab is prepared with the materials in the articles [Text Preprocessing in Python: Steps, Tools, and Examples]( https://www.kdnuggets.com/2018/11/text-preprocessing-python.html)
and [Text Data Preprocessing: A Walkthrough in Python](https://www.kdnuggets.com/2018/03/text-data-preprocessing-walkthrough-python.html)

2. We outline the basic steps of text preprocessing, which are needed for transferring text from human language to machine-readable format for further processing.  

### Coverd Topics:
* Bag-of-words Model
* Feature Selection based on IG

#### BOW vector generation
##### Count Vectors as features
Count Vector (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.

In [52]:
### Create feature vectors using CountVectorizer 
### binary parameter used to indicate word presence

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'UNC played Duke in basketball',
    'Duke lost the basketball game',
    'I ate a sandwich'
]
vectorizer = CountVectorizer(binary=True)
# Return a dense matrix representation of this CSR (Compressed Sparse Row) matrix.
# https://machinelearningmastery.com/sparse-matrices-for-machine-learning/ 
X = vectorizer.fit_transform(corpus).todense() 

print(vectorizer.vocabulary_) # Print the list of words in the vocabulary
for i, document in enumerate(corpus):  # Shows condensed version of the feature vectors
    print(document, '=', X[i])

{'unc': 9, 'played': 6, 'duke': 2, 'in': 4, 'basketball': 1, 'lost': 5, 'the': 8, 'game': 3, 'ate': 0, 'sandwich': 7}
UNC played Duke in basketball = [[0 1 1 0 1 0 1 0 0 1]]
Duke lost the basketball game = [[0 1 1 1 0 1 0 0 1 0]]
I ate a sandwich = [[1 0 0 0 0 0 0 1 0 0]]


In [55]:
print(vectorizer.fit_transform(corpus).todense())

[[0 1 1 0 1 0 1 0 0 1]
 [0 1 1 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 1 0 0]]


In [4]:
print(vectorizer.fit_transform(corpus))  #  Shows transformed numerical values (word frequency) of CSR Matrices 

  (0, 1)	1
  (0, 4)	1
  (0, 2)	1
  (0, 6)	1
  (0, 9)	1
  (1, 3)	1
  (1, 8)	1
  (1, 5)	1
  (1, 1)	1
  (1, 2)	1
  (2, 7)	1
  (2, 0)	1


In [5]:
### CountVectorizer - Creating feature vectors with frequencies of words

corpus = ['The dog ate a sandwich, the wizard transfigured a sandwich, and I ate a sandwich']
vectorizer = CountVectorizer(stop_words='english')

### Print the class model with it's parameters
print(vectorizer.fit(corpus), '\n')

### Print the list of words in the vocabulary
print(vectorizer.vocabulary_, '\n')

### Shows the vector with transformed numerical values (word frequency)
print(vectorizer.transform(corpus), '\n')

### Shows condensed version of the feature vectors
print(vectorizer.transform(corpus).todense())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None) 

{'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3} 

  (0, 0)	2
  (0, 1)	1
  (0, 2)	3
  (0, 3)	1
  (0, 4)	1 

[[2 1 3 1 1]]


#### TF-IDF Vectors as features

TF-IDF Vectors (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) can be generated.

In [6]:
### Creating TF and TF-IDF feature vectors using TfidfTransformer. 
### Note the change in IDF feature values with different number of documents in corpus

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(use_idf=False, norm=None)  # None for no normalization.
transformerIDF = TfidfTransformer(use_idf=True, norm=None)
X = vectorizer.fit_transform(corpus)
print('Vocabulary:\n', vectorizer.vocabulary_)
print('Count vectors:\n', X.todense())
print('TF vectors:\n', transformer.fit_transform(X).todense())  # divided by the document length
print('TF-IDF vectors:\n', transformerIDF.fit_transform(X).todense())

Vocabulary:
 {'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3}
Count vectors:
 [[2 1 2 0 0]
 [0 0 1 1 1]]
TF vectors:
 [[2. 1. 2. 0. 0.]
 [0. 0. 1. 1. 1.]]
TF-IDF vectors:
 [[2.81093022 1.40546511 2.         0.         0.        ]
 [0.         0.         1.         1.40546511 1.40546511]]


In [7]:
### TfidfVectorizer - combine CountVectorizer and TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = TfidfVectorizer(stop_words='english', norm=None)
print(vectorizer.fit_transform(corpus).todense(), "\n")

[[2.81093022 1.40546511 2.         0.         0.        ]
 [0.         0.         1.         1.40546511 1.40546511]] 



In [8]:
idf = vectorizer.idf_
print (dict(zip(vectorizer.get_feature_names(), idf)))

# formula : E.g., "ate" in the first column of the first document
# df = 1
# N = 2
# idf = ln((N + 1)/(df + 1)) + 1 = log(3/2) + 1 = 1.4054651081081644

# # tf = 2
# tfidf = tf * idf = 2 * 1.4054651081081644 = 2.81093022

{'ate': 1.4054651081081644, 'dog': 1.4054651081081644, 'sandwich': 1.0, 'transfigured': 1.4054651081081644, 'wizard': 1.4054651081081644}


In [9]:
# calculate normalized term vectors
from numpy import linalg as LA

doc1 = [2, 1, 2, 0, 0]
doc1_norm = LA.norm(doc1)     # Norm='l2' used to normalize term vectors.
print(doc1 / doc1_norm)

doc1 = [2.81093022, 1.40546511, 2., 0., 0.]
doc1_norm = LA.norm(doc1)     # Norm='l2' used to normalize term vectors.
print(doc1 / doc1_norm)

[0.66666667 0.33333333 0.66666667 0.         0.        ]
[0.75458397 0.37729199 0.53689271 0.         0.        ]


In [10]:
### Norm='l2' used to normalize term vectors.
### Creating TF and TF-IDF feature vectors using TfidfTransformer. 
### Note the change in IDF feature values with different number of documents in corpus

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(use_idf=False)     # Norm='l2' used to normalize term vectors.
transformerIDF = TfidfTransformer(use_idf=True)
X = vectorizer.fit_transform(corpus)
print('Vocabulary:\n', vectorizer.vocabulary_)
print('Count vectors:\n', X.todense())
print('TF vectors:\n', transformer.fit_transform(X).todense())  # divided by the document length
print('TF-IDF vectors:\n', transformerIDF.fit_transform(X).todense())

Vocabulary:
 {'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3}
Count vectors:
 [[2 1 2 0 0]
 [0 0 1 1 1]]
TF vectors:
 [[0.66666667 0.33333333 0.66666667 0.         0.        ]
 [0.         0.         0.57735027 0.57735027 0.57735027]]
TF-IDF vectors:
 [[0.75458397 0.37729199 0.53689271 0.         0.        ]
 [0.         0.         0.44943642 0.6316672  0.6316672 ]]


#### Feature Selection with Mutual Information
The classes in the [sklearn.feature_selection module](https://scikit-learn.org/stable/modules/feature_selection.html) can be used for feature selection/dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets:
[SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) and
[mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

In [11]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer

categories = ['talk.religion.misc',
              'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)

CountVectorizer(): max_df (or min_df) -> float in range [0.0, 1.0] or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold. If float, the parameter represents a proportion of documents, integer absolute counts. 

In [12]:
X, Y = newsgroups_train.data, newsgroups_train.target
cv = CountVectorizer(max_df=0.95, min_df=2,
                                     max_features=1000,
                                     stop_words='english')
X_vec = cv.fit_transform(X)

# discrete_features determines whether to consider all features discrete or continuous.
res = dict(zip(cv.get_feature_names(),
               mutual_info_classif(X_vec, Y, discrete_features=True)
               ))

In [13]:
res

{'00': 0.0062217548845825985,
 '000': 0.01130246093292954,
 '01': 0.005063992737829265,
 '04': 0.012026433047572957,
 '10': 0.011263122832501294,
 '100': 0.006615927491390547,
 '11': 0.004641632832754196,
 '12': 0.004193038975187122,
 '128': 0.006383495008360297,
 '13': 0.0026741423156522353,
 '14': 0.009988595121275111,
 '15': 0.00888293723853991,
 '16': 0.003039223390512498,
 '17': 0.003866538557383308,
 '18': 0.0055753610751617935,
 '19': 0.006818744393343739,
 '1992': 0.004722913980597322,
 '1993': 0.021535363748277622,
 '1993apr20': 0.007531285132395171,
 '20': 0.0069827829324031355,
 '200': 0.0031613172636309585,
 '2000': 0.03156572212279566,
 '21': 0.004518704154418488,
 '22': 0.0059742594539248794,
 '23': 0.008846374447591815,
 '24': 0.012164516966970542,
 '25': 0.007347126268318042,
 '256': 0.021288835018110498,
 '26': 0.002501256984401977,
 '27': 0.003937783185748935,
 '28': 0.0034713416633355794,
 '29': 0.0013711025297248802,
 '30': 0.01120496941769199,
 '300': 0.01025573654

In [14]:
# show top-10 results
print (sorted( [ (round(v,2),k) for k,v in res.items() ], reverse=True )[:10])

[(0.2, 'space'), (0.13, 'graphics'), (0.13, 'christian'), (0.12, 'god'), (0.11, 'nasa'), (0.09, 'orbit'), (0.09, 'jesus'), (0.07, 'religion'), (0.07, 'people'), (0.07, 'christians')]


In [15]:
# show bottom-10 results
print (sorted( [ (round(v,2),k) for k,v in res.items() ], reverse=True )[-10:])

[(0.0, '27'), (0.0, '26'), (0.0, '21'), (0.0, '200'), (0.0, '1992'), (0.0, '17'), (0.0, '16'), (0.0, '13'), (0.0, '12'), (0.0, '11')]


In [16]:
X_vec.shape

(1554, 1000)

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

X_new = SelectKBest(mutual_info_classif, k=100).fit_transform(X_vec, Y)  # select top 100 features
X_new.shape

(1554, 100)

In [18]:
X_new.todense()

matrix([[0, 0, 3, ..., 0, 0, 0],
        [0, 0, 2, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
import pandas as pd

# load HAM/SPAM classification example
df = pd.read_csv('../../data/sms.csv')

### Assign X=>message, y=>label
X, Y = df['message'], df['label']


In [20]:
cv = CountVectorizer(stop_words=None, lowercase=False, token_pattern=r'(?u)\b\w+\b')
X_vec = cv.fit_transform(X)

# discrete_features determines whether to consider all features discrete or continuous.
res = dict(zip(cv.get_feature_names(),
               mutual_info_classif(X_vec, Y, discrete_features=True)
               ))
res

{'0': 0.0008101570242918777,
 '00': 0.0036161266131562123,
 '000': 0.009812185545781608,
 '000pes': 2.5816587008684402e-05,
 '008704050406': 0.0007215513030074884,
 '0089': 0.00036067150309952375,
 '0121': 0.00036067150309952375,
 '01223585236': 0.00036067150309952375,
 '01223585334': 0.0007215513030074884,
 '0125698789': 2.5816587008684402e-05,
 '02': 0.0028912221496707764,
 '0207': 0.0010826397167487851,
 '02072069400': 0.00036067150309952375,
 '02073162414': 0.0007215513030074884,
 '02085076972': 0.00036067150309952375,
 '021': 0.0007215513030074884,
 '03': 0.0047050679842674335,
 '04': 0.004341875803551788,
 '0430': 0.00036067150309952375,
 '05': 0.0014439370622157804,
 '050703': 0.0007215513030074884,
 '0578': 0.0007215513030074884,
 '06': 0.0028912221496707764,
 '07': 0.0007215513030074884,
 '07008009200': 0.00036067150309952375,
 '07046744435': 0.00036067150309952375,
 '07090201529': 0.00036067150309952375,
 '07090298926': 0.00036067150309952375,
 '07099833605': 0.00036067150309

In [21]:
print (sorted( [ (round(v,2),k) for k,v in res.items() ], reverse=True )[:10])

[(0.04, 'Call'), (0.03, 'www'), (0.03, 'to'), (0.03, 'prize'), (0.03, 'or'), (0.03, 'mobile'), (0.03, 'claim'), (0.03, 'call'), (0.03, 'FREE'), (0.03, '150p')]
